In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import nltk
import numpy as np
import pandas as pd
import gensim
import transformers
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
from gensim.models import Word2Vec, word2vec
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from PIL import Image
import natsort
import os

[nltk_data] Downloading package stopwords to /home/nitesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-07-15 00:12:48.583724: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Load the data
train2 = pd.read_csv('training1.csv', delimiter='\t')

IDs = []  #set1
images = []
directory = 'TRAINING'   # directory where we have images 
filenames = natsort.natsorted(os.listdir(directory))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(filenames):
    print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        ID = filename
        pathname = os.path.join(directory, filename)
        im = Image.open(pathname)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
            print("This is 1 channel, so we omit it", imnp.shape, filename)
            continue
        IDs.append(ID)
        images.append(imnp)
        

        
def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
list1 = IDs
list2 = list(train2.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(list1, list2)
print('len of common strings', len(common_strings))



# Sort the dataframe with natural ordering of the IDs
train2['prefix_file_name'] = train2['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
sorted_df = train2.sort_values(by='prefix_file_name', ascending=True)

# Print the sorted DataFrame
# print('sorted_df shape', sorted_df.shape)


# Assuming 'df' is your DataFrame and 'common_strings' is the list of strings
# Get the common string values in the column 
filtered_df = sorted_df[sorted_df['file_name'].isin(common_strings)].reset_index(drop=True)

# Print the filtered DataFrame
print('filtered_df shape', filtered_df.shape)

train = filtered_df.copy()
print(train.shape)


# # We are just doing classification so no filename is required for our task. 
# train.drop(['file_name'], axis=1, inplace=True)
# train = train.iloc[:, [-1] + list(range(len(train.columns)-1))]
trainx = train.rename(columns={'Text Transcription': 'text'})
display(trainx)

0 1.jpg
1 2.jpg
2 3.jpg
3 4.jpg
4 5.jpg
5 6.jpg
6 7.jpg
7 8.jpg
8 9.jpg
9 10.jpg
10 11.jpg
11 12.jpg
12 13.jpg
13 14.jpg
14 15.jpg
15 16.jpg
16 17.jpg
17 18.jpg
18 19.jpg
19 21.jpg
20 22.jpg
21 23.jpg
22 24.jpg
23 25.jpg
24 26.jpg
25 27.jpg
26 28.jpg
27 30.jpg
28 32.jpg
29 33.jpg
30 34.jpg
31 35.jpg
32 36.jpg
33 37.jpg
34 39.jpg
35 40.jpg
36 42.jpg
37 43.jpg
38 44.jpg
39 45.jpg
40 47.jpg
41 48.jpg
42 50.jpg
43 52.jpg
44 53.jpg
45 55.jpg
46 56.jpg
47 57.jpg
48 58.jpg
49 59.jpg
50 60.jpg
51 62.jpg
52 63.jpg
53 64.jpg
54 65.jpg
55 66.jpg
56 67.jpg
57 69.jpg
58 70.jpg
59 71.jpg
60 73.jpg
61 74.jpg
62 75.jpg
63 78.jpg
64 79.jpg
65 80.jpg
66 81.jpg
67 82.jpg
68 83.jpg
69 87.jpg
70 89.jpg
71 90.jpg
72 91.jpg
73 92.jpg
74 93.jpg
75 94.jpg
76 96.jpg
77 97.jpg
78 98.jpg
79 99.jpg
80 101.jpg
81 102.jpg
82 103.jpg
83 104.jpg
84 105.jpg
85 106.jpg
86 107.jpg
87 108.jpg
88 109.jpg
89 111.jpg
90 112.jpg
91 113.jpg
92 114.jpg
93 115.jpg
94 116.jpg
95 117.jpg
96 118.jpg
97 119.jpg
98 120.jpg
99 121.jpg

708 850.jpg
709 851.jpg
710 852.jpg
711 853.jpg
712 854.jpg
713 855.jpg
714 856.jpg
715 857.jpg
716 858.jpg
717 860.jpg
718 862.jpg
719 864.jpg
720 865.jpg
721 866.jpg
722 867.jpg
723 869.jpg
724 870.jpg
725 871.jpg
726 872.jpg
727 873.jpg
728 874.jpg
729 876.jpg
730 877.jpg
731 878.jpg
732 880.jpg
733 881.jpg
734 882.jpg
735 883.jpg
736 885.jpg
737 886.jpg
738 887.jpg
739 888.jpg
740 889.jpg
741 890.jpg
742 891.jpg
743 892.jpg
744 893.jpg
745 895.jpg
746 896.jpg
747 897.jpg
748 898.jpg
749 900.jpg
750 901.jpg
751 903.jpg
752 905.jpg
753 906.jpg
754 907.jpg
755 908.jpg
756 910.jpg
757 911.jpg
758 912.jpg
759 914.jpg
760 915.jpg
761 917.jpg
762 919.jpg
763 921.jpg
764 922.jpg
765 923.jpg
766 924.jpg
767 925.jpg
768 926.jpg
769 928.jpg
770 929.jpg
771 930.jpg
772 931.jpg
773 932.jpg
774 934.jpg
775 937.jpg
776 939.jpg
777 940.jpg
778 941.jpg
779 942.jpg
780 943.jpg
781 944.jpg
782 945.jpg
783 946.jpg
784 947.jpg
785 949.jpg
786 950.jpg
787 951.jpg
788 952.jpg
789 953.jpg
790 954.jpg
791 

1367 1682.jpg
1368 1683.jpg
1369 1684.jpg
1370 1685.jpg
1371 1687.jpg
1372 1688.jpg
1373 1689.jpg
1374 1690.jpg
1375 1692.jpg
1376 1693.jpg
1377 1694.jpg
1378 1695.jpg
1379 1696.jpg
1380 1697.jpg
1381 1699.jpg
1382 1700.jpg
1383 1701.jpg
1384 1702.jpg
1385 1703.jpg
1386 1705.jpg
1387 1706.jpg
1388 1707.jpg
1389 1709.jpg
1390 1710.jpg
1391 1711.jpg
1392 1713.jpg
1393 1714.jpg
1394 1715.jpg
1395 1716.jpg
1396 1717.jpg
1397 1718.jpg
1398 1720.jpg
1399 1721.jpg
1400 1722.jpg
1401 1723.jpg
1402 1724.jpg
1403 1725.jpg
1404 1726.jpg
1405 1727.jpg
1406 1728.jpg
1407 1731.jpg
1408 1732.jpg
1409 1733.jpg
1410 1734.jpg
1411 1735.jpg
1412 1736.jpg
1413 1737.jpg
1414 1738.jpg
1415 1739.jpg
1416 1740.jpg
1417 1742.jpg
1418 1743.jpg
1419 1744.jpg
1420 1745.jpg
1421 1746.jpg
1422 1747.jpg
1423 1748.jpg
1424 1749.jpg
1425 1750.jpg
1426 1751.jpg
1427 1752.jpg
1428 1754.jpg
1429 1755.jpg
1430 1756.jpg
1431 1757.jpg
1432 1758.jpg
1433 1759.jpg
1434 1760.jpg
1435 1762.jpg
1436 1763.jpg
1437 1764.jpg
1438 1

1975 2430.jpg
1976 2431.jpg
1977 2432.jpg
1978 2433.jpg
1979 2434.jpg
1980 2435.jpg
1981 2436.jpg
This is 1 channel, so we omit it (224, 224) 2436.jpg
1982 2437.jpg
1983 2438.jpg
1984 2439.jpg
1985 2440.jpg
1986 2441.jpg
1987 2444.jpg
1988 2446.jpg
1989 2447.jpg
1990 2448.jpg
1991 2449.jpg
1992 2450.jpg
1993 2453.jpg
1994 2454.jpg
1995 2455.jpg
1996 2456.jpg
1997 2457.jpg
1998 2458.jpg
1999 2459.jpg
2000 2460.jpg
2001 2461.jpg
2002 2463.jpg
2003 2464.jpg
2004 2465.jpg
2005 2466.jpg
2006 2467.jpg
2007 2468.jpg
2008 2469.jpg
2009 2471.jpg
2010 2472.jpg
2011 2473.jpg
2012 2474.jpg
2013 2475.jpg
2014 2476.jpg
2015 2477.jpg
2016 2478.jpg
2017 2479.jpg
2018 2481.jpg
2019 2483.jpg
2020 2484.jpg
2021 2485.jpg
2022 2486.jpg
2023 2487.jpg
2024 2490.jpg
2025 2491.jpg
2026 2492.jpg
2027 2493.jpg
2028 2494.jpg
2029 2496.jpg
2030 2497.jpg
2031 2498.jpg
2032 2500.jpg
2033 2501.jpg
2034 2502.jpg
2035 2503.jpg
2036 2504.jpg
2037 2505.jpg
2038 2506.jpg
2039 2508.jpg
2040 2509.jpg
2041 2510.jpg
2042 2511

2586 3189.jpg
2587 3190.jpg
2588 3191.jpg
2589 3192.jpg
2590 3193.jpg
2591 3195.jpg
2592 3198.jpg
2593 3199.jpg
2594 3200.jpg
2595 3201.jpg
2596 3202.jpg
2597 3203.jpg
2598 3205.jpg
2599 3206.jpg
2600 3208.jpg
2601 3209.jpg
2602 3210.jpg
2603 3211.jpg
2604 3212.jpg
2605 3214.jpg
2606 3215.jpg
2607 3216.jpg
2608 3218.jpg
2609 3219.jpg
2610 3220.jpg
2611 3221.jpg
2612 3222.jpg
2613 3223.jpg
2614 3224.jpg
2615 3225.jpg
2616 3227.jpg
2617 3228.jpg
2618 3229.jpg
2619 3230.jpg
2620 3232.jpg
2621 3233.jpg
2622 3234.jpg
2623 3235.jpg
2624 3236.jpg
2625 3237.jpg
2626 3238.jpg
2627 3239.jpg
2628 3240.jpg
2629 3241.jpg
2630 3243.jpg
2631 3244.jpg
2632 3246.jpg
2633 3247.jpg
2634 3248.jpg
2635 3251.jpg
2636 3252.jpg
2637 3253.jpg
2638 3254.jpg
2639 3255.jpg
2640 3256.jpg
2641 3257.jpg
2642 3258.jpg
2643 3259.jpg
2644 3260.jpg
2645 3262.jpg
2646 3263.jpg
2647 3264.jpg
2648 3265.jpg
2649 3266.jpg
2650 3267.jpg
2651 3268.jpg
2652 3269.jpg
2653 3272.jpg
2654 3274.jpg
2655 3276.jpg
2656 3277.jpg
2657 3

...

8054 9682.jpg
8055 9683.jpg
8056 9684.jpg
8057 9685.jpg
8058 9687.jpg
8059 9688.jpg
8060 9689.jpg
8061 9690.jpg
8062 9691.jpg
8063 9692.jpg
8064 9693.jpg
8065 9694.jpg
8066 9696.jpg
8067 9698.jpg
8068 9699.jpg
8069 9700.jpg
8070 9701.jpg
8071 9703.jpg
8072 9704.jpg
8073 9705.jpg
8074 9706.jpg
8075 9707.jpg
8076 9708.jpg
8077 9709.jpg
8078 9710.jpg
8079 9711.jpg
8080 9712.jpg
8081 9713.jpg
8082 9714.jpg
8083 9716.jpg
8084 9717.jpg
8085 9718.jpg
8086 9719.jpg
8087 9720.jpg
8088 9721.jpg
8089 9722.jpg
8090 9723.jpg
8091 9724.jpg
8092 9725.jpg
8093 9726.jpg
8094 9727.jpg
8095 9728.jpg
8096 9729.jpg
8097 9730.jpg
8098 9732.jpg
8099 9733.jpg
8100 9734.jpg
8101 9735.jpg
8102 9737.jpg
8103 9738.jpg
8104 9739.jpg
8105 9740.jpg
8106 9741.jpg
8107 9743.jpg
8108 9744.jpg
8109 9745.jpg
8110 9746.jpg
8111 9747.jpg
8112 9748.jpg
8113 9749.jpg
8114 9750.jpg
8115 9751.jpg
8116 9752.jpg
8117 9753.jpg
8118 9754.jpg
8119 9755.jpg
8120 9756.jpg
8121 9757.jpg
8122 9759.jpg
8123 9760.jpg
8124 9761.jpg
8125 9

8658 10380.jpg
8659 10381.jpg
8660 10382.jpg
8661 10383.jpg
8662 10384.jpg
8663 10385.jpg
8664 10386.jpg
8665 10387.jpg
8666 10388.jpg
8667 10389.jpg
8668 10392.jpg
8669 10393.jpg
8670 10395.jpg
8671 10396.jpg
8672 10399.jpg
8673 10400.jpg
8674 10402.jpg
8675 10403.jpg
8676 10404.jpg
8677 10405.jpg
8678 10406.jpg
8679 10407.jpg
8680 10408.jpg
8681 10409.jpg
8682 10410.jpg
8683 10411.jpg
8684 10412.jpg
8685 10413.jpg
8686 10415.jpg
8687 10416.jpg
8688 10417.jpg
8689 10418.jpg
8690 10419.jpg
8691 10421.jpg
8692 10423.jpg
8693 10424.jpg
8694 10425.jpg
8695 10426.jpg
8696 10427.jpg
8697 10428.jpg
8698 10429.jpg
8699 10430.jpg
8700 10432.jpg
8701 10433.jpg
8702 10434.jpg
8703 10435.jpg
8704 10436.jpg
8705 10437.jpg
8706 10438.jpg
8707 10439.jpg
8708 10440.jpg
8709 10441.jpg
8710 10442.jpg
8711 10443.jpg
8712 10444.jpg
8713 10446.jpg
8714 10447.jpg
8715 10448.jpg
8716 10449.jpg
8717 10450.jpg
8718 10451.jpg
8719 10452.jpg
8720 10453.jpg
8721 10454.jpg
8722 10455.jpg
8723 10457.jpg
8724 10458

9243 11074.jpg
9244 11075.jpg
9245 11077.jpg
9246 11078.jpg
9247 11079.jpg
9248 11080.jpg
9249 11081.jpg
9250 11082.jpg
9251 11083.jpg
9252 11084.jpg
9253 11085.jpg
9254 11086.jpg
9255 11087.jpg
9256 11088.jpg
9257 11089.jpg
9258 11090.jpg
9259 11092.jpg
9260 11094.jpg
9261 11095.jpg
9262 11096.jpg
9263 11097.jpg
9264 11098.jpg
9265 11099.jpg
9266 11101.jpg
9267 11102.jpg
9268 11103.jpg
9269 11104.jpg
9270 11105.jpg
9271 11107.jpg
9272 11108.jpg
9273 11109.jpg
9274 11112.jpg
9275 11113.jpg
9276 11114.jpg
9277 11115.jpg
9278 11116.jpg
9279 11117.jpg
9280 11118.jpg
9281 11119.jpg
9282 11120.jpg
9283 11122.jpg
9284 11123.jpg
9285 11124.jpg
9286 11125.jpg
9287 11126.jpg
9288 11127.jpg
9289 11128.jpg
9290 11129.jpg
9291 11130.jpg
9292 11131.jpg
9293 11132.jpg
9294 11133.jpg
9295 11134.jpg
9296 11136.jpg
9297 11137.jpg
9298 11138.jpg
9299 11140.jpg
9300 11141.jpg
9301 11142.jpg
9302 11143.jpg
9303 11144.jpg
9304 11145.jpg
9305 11146.jpg
9306 11147.jpg
9307 11148.jpg
9308 11151.jpg
9309 11152

9789 11713.jpg
9790 11714.jpg
9791 11715.jpg
This is 1 channel, so we omit it (224, 224) 11715.jpg
9792 11716.jpg
9793 11717.jpg
9794 11718.jpg
9795 11719.jpg
9796 11720.jpg
9797 11721.jpg
9798 11722.jpg
9799 11723.jpg
9800 11724.jpg
9801 13000.jpg
9802 13003.jpg
9803 13006.jpg
9804 13009.jpg
9805 13018.jpg
9806 13021.jpg
9807 13027.jpg
9808 13033.jpg
9809 13036.jpg
9810 13039.jpg
9811 13042.jpg
9812 13045.jpg
9813 13051.jpg
9814 13054.jpg
9815 13057.jpg
9816 13060.jpg
9817 13063.jpg
9818 13066.jpg
9819 13069.jpg
9820 13072.jpg
9821 13075.jpg
9822 13078.jpg
9823 13081.jpg
9824 13084.jpg
9825 13090.jpg
9826 13096.jpg
9827 13099.jpg
9828 13102.jpg
9829 13105.jpg
9830 13108.jpg
9831 13111.jpg
9832 13114.jpg
9833 13117.jpg
9834 13120.jpg
9835 13123.jpg
9836 13126.jpg
9837 13129.jpg
9838 13132.jpg
9839 13135.jpg
9840 13138.jpg
9841 13141.jpg
9842 13144.jpg
9843 13147.jpg
9844 13153.jpg
9845 13156.jpg
9846 13159.jpg
9847 13162.jpg
9848 13165.jpg
9849 13168.jpg
9850 13174.jpg
9851 13177.jpg
9

,file_name,misogynous,shaming,stereotype,objectification,violence,text,prefix_file_name
0,1.jpg,0,0,0,0,0,Milk Milk.zip,1
1,2.jpg,0,0,0,0,0,-What are you doing? -you told me to satanize ...,2
2,3.jpg,0,0,0,0,0,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,3
3,4.jpg,0,0,0,0,0,Bedroom Kitchen Bathroom Bron memes storage,4
4,5.jpg,0,0,0,0,0,WAKEUP EARLY FREELANCERS,5
...,...,...,...,...,...,...,...,...
9981,15002.jpg,0,0,0,0,0,WAITING FOR THE END OF THE COVID imgflip.com,15002
9982,15003.jpg,0,0,0,0,0,SMART WOMEN ARE AROUND imgflip.com,15003
9983,15004.jpg,0,0,0,0,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,15004
9984,15005.jpg,0,0,0,0,0,COOKING FOR MY WIFE imgflip.com,15005


In [3]:
test1 = pd.read_csv('Test.csv', delimiter='\t')
test_labels = pd.read_csv('test_labels.txt', 
                          delimiter='\t',
                         header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

test = pd.merge(test1, test_labels, on='file_name', how='inner')

# train.drop(['file_name'], axis=1, inplace=True)
test.drop(['file_name'], axis=1, inplace=True)
# train = train.iloc[:, [-1] + list(range(len(train.columns)-1))]


# train = train.rename(columns={'Text Transcription': 'text'})
test2 = test.rename(columns={'Text Transcription': 'text'})
test2

,text,misogynous,shaming,stereotype,objectification,violence
0,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...,0,0,0,0,0
1,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?",1,0,1,1,0
2,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN,0,0,0,0,0
3,Photographing something you want to show every...,1,0,1,1,0
4,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...,0,0,0,0,0
...,...,...,...,...,...,...
995,IT'S NOT YOUR FAULT You didn't design the dres...,1,0,1,1,0
996,THINK ABOUT HOW MUCH BETTER HER SKIN IS BREATH...,0,0,0,0,0
997,THE STEREOTYPES ARE TRUE F SHE DOES HAVE A TIG...,1,0,1,1,0
998,DRAWS NAKED PICTURES OF BLACK WOMEN 00 0000 GE...,0,0,0,0,0


In [4]:
def export_classification_report(report, mod, exec_time):
    lines = report.split('\n')
    data = lines[2:4] + lines[5:9]
    data = [line.split() for line in data]
    

    acc = float(data[2][1])
    m_prec = float(data[3][2])
    m_recall = float(data[3][3])
    m_f1 = float(data[3][4])
    w_prec = float(data[4][2])
    w_recall = float(data[4][3])
    w_f1 = float(data[4][4])
    
    df = pd.DataFrame({
        'Model': [mod],
        'Accuracy': [acc],
        'M-Precision': [m_prec],
        'M-Recall': [m_recall],
        'M-F1-Score': [m_f1],
        'W-Precision': [w_prec],
        'W-Recall': [w_recall],
        'W-F1-Score': [w_f1],
        'Runtime': [exec_time]
    })
    
    return df

In [5]:
import torch
import time
# Set the random seed for NumPy and PyTorch


# df = pd.DataFrame(columns=['Model', 'Accuracy', 
#                            'M-Precision', 'M-Recall', 
#                            'M-F1-Score', 'W-Precision',
#                            'W-Recall', 'W-F1-Score'])

df=pd.DataFrame(columns=['Model', 'Accuracy', 
                       'M-Precision', 'M-Recall', 
                       'M-F1-Score', 'W-Precision',
                       'W-Recall', 'W-F1-Score', 'Runtime'])



# for i in list([7,8,9,10,11]):
    
for i in list([7,10,18,22,55,77]):
    
    start_time = time.time()
        
    print('bootstrap number: ', i)
    
    np.random.seed(i)
    torch.manual_seed(i)


    # device = "cuda:0" if torch.cuda.is_available() else "cpu"
    device = 'cpu'

    train = trainx[['text', 'misogynous']]
    train = train.rename(columns = {'misogynous':'label'})
    train

    test = test2[['text', 'misogynous']]
    test = test.rename(columns = {'misogynous':'label'})
    test


    train['label'] = train['label'].map({0: 'non_misogyn', 1: 'misogyn'})
    test['label'] = test['label'].map({0: 'non_misogyn', 1: 'misogyn'})




    # # # Label Encoding
    from sklearn.preprocessing import LabelEncoder
    labelencoder = LabelEncoder()

    labelencoder.fit(train['label'])
    train['encoded_label'] = labelencoder.fit_transform(train['label'])
    test["encoded_label"] = labelencoder.transform(test['label'])

    # # # Validate the mapping:
    train.groupby(["label", "encoded_label"]).aggregate("count")

    labelencoder.inverse_transform([0,1])


    import torch

    train_labels = torch.tensor(train["encoded_label"].tolist())
    test_labels = torch.tensor(test["encoded_label"].tolist())
    train_labels


    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


    # # Encoding a sentence
    # sent = "The quick brown fox jumped over the lazy dog."
    # print(f"Tokenizer output a dictionary: {tokenizer(sent)}")

    # # We can also decode ids to vocabulary
    # print(tokenizer.decode([101, 1996, 4248, 2829, 4419, 5598, 2058, 1996, 13971, 3899, 1012, 102]))


    train_encodings = tokenizer(
        train["text"].tolist(),
        padding=True,           # pad all inputs to max length
        max_length=80,         # Bert max is 512, we choose 24 for computational efficiency
        return_tensors="pt",    # Return format pytorch tensor
        truncation=True
    )

    print('train encodings shape ', train_encodings['input_ids'].shape)


    test_encodings = tokenizer(
        test["text"].tolist(),
        padding=True,           # pad all inputs to max length
        max_length=80,         # Bert max is 512, we choose 24 for computational efficiency
        return_tensors="pt",    # Return format pytorch tensor
        truncation=True
    )

    print('test encodings shape ', test_encodings['input_ids'].shape)


    from torch.utils.data import Dataset
    import torch
    from sklearn.preprocessing import LabelEncoder

    # Define Custom Class for DistilBert Inputs
    class RelationDataset(Dataset):

        def __init__(self, encodings: dict):  
            self.encodings = encodings

        def __len__(self) -> int:
            return len(self.encodings["input_ids"])

        def __getitem__(self, idx: int) -> dict:
            e = {k: v[idx] for k,v in self.encodings.items()}
            return e 


    # Update encodings with labels
    train_encodings["labels"] = train_labels
    test_encodings["labels"] = test_labels

    # Generate Datasets
    train_ds = RelationDataset(train_encodings)
    test_ds = RelationDataset(test_encodings)


    from transformers import AutoModelForSequenceClassification
    from transformers import Trainer
    from transformers import TrainingArguments

    model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

    # Freeze embeddings
    for name, param in model.distilbert.embeddings.named_parameters():
        param.requires_grad = False
        print(name, param.requires_grad)

    # Freeze layers 1-4
    freeze_layers = [1,2,3,4]
    for name, param in model.distilbert.transformer.layer.named_parameters():
        if int(name[0]) in freeze_layers:    
            param.requires_grad = False
        print(name, param.requires_grad)


    '''
    Training
    We're finally ready to train the model.
    To train you need create training arguments object which contains the training details
    like num_epochs, learning rate scheduling, batch size, etc.

    The Trainer object handles the training loop setup and training.
    '''

    from transformers import TrainingArguments
    from transformers import Trainer


    training_args = TrainingArguments(
        output_dir="./distilbert_results",
        num_train_epochs=5,
        save_strategy="epoch",
        lr_scheduler_type='cosine',
        per_device_train_batch_size = 32,
        fp16=True,
    )

    trainer = Trainer(
        model,
        training_args,
        train_dataset=train_ds,
    )

    trainer.train()


    import numpy as np
    from sklearn.metrics import classification_report 

    preds = trainer.predict(test_ds)
#     print(preds)


    preds = labelencoder.inverse_transform(np.argmax(preds.predictions, axis=1))
    print(classification_report(test["label"].tolist(), preds))


    report_distilbert=classification_report(test["label"].tolist(), preds, digits=3)
    # print(classification_report(test["label"].tolist(), preds))
    end_time = time.time()
    execution_time1 = end_time - start_time

    print(f"Execution time: {execution_time1} seconds")
    
    modname = 'DistilBERT'
    df1 = export_classification_report(report_distilbert, modname, execution_time1)
    df = pd.concat([df, df1])




bootstrap number:  7
train encodings shape  torch.Size([9986, 80])
test encodings shape  torch.Size([1000, 80])


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

word_embeddings.weight False
position_embeddings.weight False
LayerNorm.weight False
LayerNorm.bias False
0.attention.q_lin.weight True
0.attention.q_lin.bias True
0.attention.k_lin.weight True
0.attention.k_lin.bias True
0.attention.v_lin.weight True
0.attention.v_lin.bias True
0.attention.out_lin.weight True
0.attention.out_lin.bias True
0.sa_layer_norm.weight True
0.sa_layer_norm.bias True
0.ffn.lin1.weight True
0.ffn.lin1.bias True
0.ffn.lin2.weight True
0.ffn.lin2.bias True
0.output_layer_norm.weight True
0.output_layer_norm.bias True
1.attention.q_lin.weight False
1.attention.q_lin.bias False
1.attention.k_lin.weight False
1.attention.k_lin.bias False
1.attention.v_lin.weight False
1.attention.v_lin.bias False
1.attention.out_lin.weight False
1.attention.out_lin.bias False
1.sa_layer_norm.weight False
1.sa_layer_norm.bias False
1.ffn.lin1.weight False
1.ffn.lin1.bias False
1.ffn.lin2.weight False
1.ffn.lin2.bias False
1.output_layer_norm.weight False
1.output_layer_norm.bias Fals

Using amp half precision backend
***** Running training *****
  Num examples = 9986
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1565


Step,Training Loss
500,0.470500
1000,0.318400
1500,0.240000


...

word_embeddings.weight False
position_embeddings.weight False
LayerNorm.weight False
LayerNorm.bias False
0.attention.q_lin.weight True
0.attention.q_lin.bias True
0.attention.k_lin.weight True
0.attention.k_lin.bias True
0.attention.v_lin.weight True
0.attention.v_lin.bias True
0.attention.out_lin.weight True
0.attention.out_lin.bias True
0.sa_layer_norm.weight True
0.sa_layer_norm.bias True
0.ffn.lin1.weight True
0.ffn.lin1.bias True
0.ffn.lin2.weight True
0.ffn.lin2.bias True
0.output_layer_norm.weight True
0.output_layer_norm.bias True
1.attention.q_lin.weight False
1.attention.q_lin.bias False
1.attention.k_lin.weight False
1.attention.k_lin.bias False
1.attention.v_lin.weight False
1.attention.v_lin.bias False
1.attention.out_lin.weight False
1.attention.out_lin.bias False
1.sa_layer_norm.weight False
1.sa_layer_norm.bias False
1.ffn.lin1.weight False
1.ffn.lin1.bias False
1.ffn.lin2.weight False
1.ffn.lin2.bias False
1.output_layer_norm.weight False
1.output_layer_norm.bias Fals

Step,Training Loss
500,0.469600
1000,0.319500
1500,0.240100


Saving model checkpoint to ./distilbert_results/checkpoint-313
Configuration saved in ./distilbert_results/checkpoint-313/config.json
Model weights saved in ./distilbert_results/checkpoint-313/pytorch_model.bin
Saving model checkpoint to ./distilbert_results/checkpoint-626
Configuration saved in ./distilbert_results/checkpoint-626/config.json
Model weights saved in ./distilbert_results/checkpoint-626/pytorch_model.bin
Saving model checkpoint to ./distilbert_results/checkpoint-939
Configuration saved in ./distilbert_results/checkpoint-939/config.json
Model weights saved in ./distilbert_results/checkpoint-939/pytorch_model.bin
Saving model checkpoint to ./distilbert_results/checkpoint-1252
Configuration saved in ./distilbert_results/checkpoint-1252/config.json
Model weights saved in ./distilbert_results/checkpoint-1252/pytorch_model.bin
Saving model checkpoint to ./distilbert_results/checkpoint-1565
Configuration saved in ./distilbert_results/checkpoint-1565/config.json
Model weights sav

              precision    recall  f1-score   support

     misogyn       0.60      0.81      0.69       500
 non_misogyn       0.71      0.47      0.56       500

    accuracy                           0.64      1000
   macro avg       0.66      0.64      0.63      1000
weighted avg       0.66      0.64      0.63      1000

Execution time: 82.13578915596008 seconds
bootstrap number:  77


ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/distilbert-base-uncased (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f6a2a2724c0>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [6]:
display(df)

,Model,Accuracy,M-Precision,M-Recall,M-F1-Score,W-Precision,W-Recall,W-F1-Score,Runtime
0,DistilBERT,0.648,0.667,0.648,0.638,0.667,0.648,0.638,77.703677
0,DistilBERT,0.645,0.665,0.645,0.634,0.665,0.645,0.634,78.194569
0,DistilBERT,0.644,0.662,0.644,0.634,0.662,0.644,0.634,80.748714
0,DistilBERT,0.643,0.663,0.643,0.631,0.663,0.643,0.631,81.037695
0,DistilBERT,0.637,0.655,0.637,0.626,0.655,0.637,0.626,82.135789


In [7]:
df.to_excel('2_classification_report_DistilBERT.xlsx', index=False)